# Julia code for ACE basis evaluation

Setting the input parameters and variables _via_ JSON

In [5]:
using ArgParse
include("descriptor.jl")
include("trajectory.jl")

┌ Info: Precompiling ACE1pack [8c4e8d19-0bd6-4234-8309-7210652e3178]
└ @ Base loading.jl:1423
┌ Info: Skipping precompilation since __precompile__(false). Importing ACE1pack [8c4e8d19-0bd6-4234-8309-7210652e3178].
└ @ Base loading.jl:1124
┌ Info: Precompiling IPFitting [3002bd4c-79e4-52ce-b924-91256dde4e52]
└ @ Base loading.jl:1423
┌ Info: Skipping precompilation since __precompile__(false). Importing IPFitting [3002bd4c-79e4-52ce-b924-91256dde4e52].
└ @ Base loading.jl:1124
┌ Info: Precompiling ASE [51974c44-a7ed-5088-b8be-3e78c8ba416c]
└ @ Base loading.jl:1423
┌ Info: Skipping precompilation since __precompile__(false). Importing ASE [51974c44-a7ed-5088-b8be-3e78c8ba416c].
└ @ Base loading.jl:1124


_traj_init (generic function with 1 method)

In [6]:
# full path not necessary if .json file is in the same dir
#input_dict = JSON.parsefile("evACE.json")
input_dict = load_dict("evACE.json")

Dict{String, Any} with 3 entries:
  "evaluation" => Dict{String, Any}("Z"=>3)
  "ace_basis"  => Dict{String, Any}("rcut"=>5.5, "N"=>3, "maxdeg"=>4, "species"…
  "trajectory" => Dict{String, Any}("traj_file"=>"./examples/traj_2.1_0-100-1_n…

## init the Evaluation

In [9]:
# read first frame
traj_init = _traj_init(input_dict["trajectory"])

┌ Info: Keys used: E => "", F => "", V => ""
└ @ IPFitting.Data /home/agardin/.julia/packages/IPFitting/E09UD/src/data.jl:152


┌─────────────┬───────┬───────┬───────┬───────┬───────┐
│ config_type │ #cfgs │ #envs │    #E │    #F │    #V │
│      String │ Int64 │ Int64 │ Int64 │ Int64 │ Int64 │
├─────────────┼───────┼───────┼───────┼───────┼───────┤
│     nothing │     1 │  8402 │     0 │     0 │     0 │
├─────────────┼───────┼───────┼───────┼───────┼───────┤
│       total │     1 │  8402 │     0 │     0 │     0 │
│     missing │     0 │     0 │     1 │ 25206 │     9 │
└─────────────┴───────┴───────┴───────┴───────┴───────┘


1-element Vector{Dat}:
 Dat(Atoms{Float64}(StaticArraysCore.SVector{3, Float64}[[13.4214334, 37.9411774, 25.6024189], [12.4229565, 39.045311, 25.9095592], [14.2894106, 38.552597, 24.6148567], [12.5934677, 39.9337463, 24.7763767], [13.7624607, 39.6994514, 24.1646919], [12.8664417, 37.092144, 25.1768265], [14.003438, 37.619648, 26.4787102], [12.6682644, 39.5936661, 26.8310394], [11.3863182, 38.6841393, 25.8393097], [14.3525457, 40.5229683, 23.5030918]  …  [28.7237568, 37.9477577, 18.0548611], [16.9131069, 28.2505417, 28.0804462], [44.8431015, 22.4488029, 19.3294239], [40.0488434, 42.6378517, 27.3642979], [18.8909836, 43.4313812, 1.53649151], [30.3799152, 30.5201054, 44.2985649], [21.5288963, 0.73614603, 40.0230713], [8.79868984, 37.8729858, 21.1588421], [44.5085945, 33.5317574, 24.1566582], [35.9943466, 7.74214506, 11.2953444]], StaticArraysCore.SVector{3, Float64}[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0,

### ACE basis init

In [10]:
input_dict["ace_basis"]

Dict{String, Any} with 4 entries:
  "rcut"    => 5.5
  "N"       => 3
  "maxdeg"  => 5
  "species" => Any["H"]

In [11]:
# -- String to Symbol
_params_to_species(species::Union{AbstractArray{T}, Tuple{T, T}}) where T <: AbstractString  = 
      Symbol.(species)

_params_to_species(dict::Dict{Tuple{Tsym, Tsym}, Tval}) where Tsym <: AbstractString where Tval <: Any = 
      Dict(Tuple(_params_to_species(d)) => val for (d, val) in dict)

_params_to_species(dict::Nothing) = nothing


# -- Build the ace basis (rpi or ace ??)
function dict_to_ace_basis(a::Dict)
    # species
    a["species"] = convert(Vector{String}, a["species"])
    species = _params_to_species(a["species"])
    # body order
    N = a["N"]
    # max poly degree
    maxdeg = a["maxdeg"]
    # rcut
    rcut = a["rcut"]

    # -- Default paramters
    r0 = get!(input_dict["ace_basis"], "r0", 0.5)
    pin = get!(input_dict["ace_basis"], "pin", 2)

    # -- Hidden parameters
    # degree - set as default as initial setp
    _rin = 0.1
    _D = SparsePSHDegree(; wL=1.5, csp=1.0)

    return ACE1pack.ACE1.Utils.rpi_basis(;
            species = species,
            N = N,
            maxdeg = maxdeg,
            D = _D,
            rcut = rcut, r0 = r0, rin = _rin,
            pin = pin
            )
end

dict_to_ace_basis (generic function with 1 method)

In [13]:
ace_basis = dict_to_ace_basis(input_dict["ace_basis"])

RPIBasis{Float64, BasicPSH1pBasis{Float64, 1, ACE1.OrthPolys.TransformedPolys{Float64, PolyTransform{Int64, Float64}, ACE1.OrthPolys.OrthPolyBasis{Float64}}}, 1, ACE1.DAGEvaluator}(PIBasis{BasicPSH1pBasis{Float64, 1, ACE1.OrthPolys.TransformedPolys{Float64, PolyTransform{Int64, Float64}, ACE1.OrthPolys.OrthPolyBasis{Float64}}}, 1, ACE1.DAGEvaluator}(BasicPSH1pBasis{Float64, 1, ACE1.OrthPolys.TransformedPolys{Float64, PolyTransform{Int64, Float64}, ACE1.OrthPolys.OrthPolyBasis{Float64}}}(ACE1.OrthPolys.TransformedPolys{Float64, PolyTransform{Int64, Float64}, ACE1.OrthPolys.OrthPolyBasis{Float64}}(ACE1.OrthPolys.OrthPolyBasis{Float64}(2, 0.053254437869822494, 2, 1.8595041322314048, [2.358083222149332, 3.6723879325307336, 2.9607738063804043, 2.6916125512549365, 2.5521234650096187], [0.0, -3.512195745342244, -2.831622736142675, -2.5742024874024536, -2.4407980148267954], [0.0, 0.0, -0.8062257748298562, -0.9090909090909164, -0.9481763873554993], [0.05415756271700328, 0.05596381241136487, 0.0

In [14]:
length(ace_basis)

16